In [2]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train', shuffle = True)
newsgroups_test = fetch_20newsgroups(subset='test', shuffle = True)

In [3]:
# Let's see the topics
print(list(newsgroups_train.target_names))

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [12]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [13]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/hardp/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [23]:
# Test of lemmatizer 
# print(WordNetLemmatizer().lemmatize('went', pos = 'v')) # past tense to present tense

In [24]:
# Stemmer Example
import pandas as pd
stemmer = SnowballStemmer("english")
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]

pd.DataFrame(data={'original word':original_words, 'stemmed':singles })

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [25]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [27]:
'''
Preview a document after preprocessing
'''
document_num = 50
doc_sample = 'Bob Hawke, Australia\'s charismatic labor prime minister for much of the 1980s, dies aged 89'

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['Bob', 'Hawke,', "Australia's", 'charismatic', 'labor', 'prime', 'minister', 'for', 'much', 'of', 'the', '1980s,', 'dies', 'aged', '89']


Tokenized and lemmatized document: 
['hawk', 'australia', 'charismat', 'labor', 'prime', 'minist', 'die', 'age']


In [31]:
# Preprocess all the headlines we have 
processed_docs = []

for doc in newsgroups_train.data:
    processed_docs.append(preprocess(doc))

In [32]:
'''
Preview 'processed_docs'
'''
print(processed_docs[:2])

[['lerxst', 'thing', 'subject', 'nntp', 'post', 'host', 'organ', 'univers', 'maryland', 'colleg', 'park', 'line', 'wonder', 'enlighten', 'door', 'sport', 'look', 'late', 'earli', 'call', 'bricklin', 'door', 'small', 'addit', 'bumper', 'separ', 'rest', 'bodi', 'know', 'tellm', 'model', 'engin', 'spec', 'year', 'product', 'histori', 'info', 'funki', 'look', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst'], ['guykuo', 'carson', 'washington', 'subject', 'clock', 'poll', 'final', 'summari', 'final', 'clock', 'report', 'keyword', 'acceler', 'clock', 'upgrad', 'articl', 'shelley', 'qvfo', 'innc', 'organ', 'univers', 'washington', 'line', 'nntp', 'post', 'host', 'carson', 'washington', 'fair', 'number', 'brave', 'soul', 'upgrad', 'clock', 'oscil', 'share', 'experi', 'poll', 'send', 'brief', 'messag', 'detail', 'experi', 'procedur', 'speed', 'attain', 'rat', 'speed', 'card', 'adapt', 'heat', 'sink', 'hour', 'usag', 'floppi', 'disk', 'function', 'floppi', 'especi', 'request', 'summar', 'day',

In [38]:
# Steps to find number of times a word appeared in dictionary

'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''
dictionary = gensim.corpora.Dictionary(processed_docs)

11314


In [44]:
'''
Checking dictionary created
'''
# print(len(dictionary.items())) - around 61411 items
count = 0
for k, v in dictionary.items():
    print(k, v)
    count += 1
    if count > 10:
        break

0 addit
1 bodi
2 bricklin
3 bring
4 bumper
5 call
6 colleg
7 door
8 earli
9 engin
10 enlighten


In [45]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [46]:
'''
Preview BOW for our sample preprocessed document
'''
document_num = 20
bow_doc_x = bow_corpus[document_num]

for i in range(len(bow_doc_x)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_x[i][0], 
                                                     dictionary[bow_doc_x[i][0]], 
                                                     bow_doc_x[i][1]))

Word 13 ("host") appears 1 time.
Word 18 ("line") appears 1 time.
Word 24 ("nntp") appears 1 time.
Word 25 ("organ") appears 1 time.
Word 27 ("post") appears 1 time.
Word 29 ("rest") appears 1 time.
Word 34 ("subject") appears 1 time.
Word 37 ("thing") appears 5 time.
Word 115 ("give") appears 1 time.
Word 128 ("like") appears 1 time.
Word 138 ("peopl") appears 1 time.
Word 212 ("write") appears 1 time.
Word 221 ("clear") appears 1 time.
Word 338 ("say") appears 1 time.
Word 386 ("think") appears 1 time.
Word 437 ("refer") appears 1 time.
Word 452 ("true") appears 1 time.
Word 504 ("technolog") appears 1 time.
Word 564 ("christian") appears 1 time.
Word 582 ("exampl") appears 1 time.
Word 607 ("jew") appears 1 time.
Word 612 ("lead") appears 1 time.
Word 615 ("littl") appears 3 time.
Word 664 ("wors") appears 2 time.
Word 920 ("keith") appears 3 time.
Word 933 ("punish") appears 1 time.
Word 1016 ("california") appears 1 time.
Word 1083 ("institut") appears 1 time.
Word 1147 ("similar"

In [47]:
# LDA mono-core -- fallback code in case LdaMulticore throws an error on your machine
# lda_model = gensim.models.LdaModel(bow_corpus, 
#                                    num_topics = 10, 
#                                    id2word = dictionary,                                    
#                                    passes = 50)

# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'
'''
# TODO
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 8, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [48]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.012*"line" + 0.012*"subject" + 0.012*"organ" + 0.011*"write" + 0.011*"articl" + 0.009*"space" + 0.007*"post" + 0.006*"univers" + 0.006*"nasa" + 0.005*"host"


Topic: 1 
Words: 0.011*"game" + 0.010*"line" + 0.009*"team" + 0.009*"subject" + 0.009*"organ" + 0.008*"year" + 0.007*"write" + 0.007*"play" + 0.006*"player" + 0.006*"articl"


Topic: 2 
Words: 0.008*"christian" + 0.008*"write" + 0.008*"think" + 0.008*"peopl" + 0.007*"subject" + 0.007*"know" + 0.006*"line" + 0.006*"believ" + 0.006*"say" + 0.006*"organ"


Topic: 3 
Words: 0.015*"line" + 0.014*"subject" + 0.013*"organ" + 0.008*"post" + 0.008*"drive" + 0.008*"univers" + 0.008*"window" + 0.007*"host" + 0.007*"nntp" + 0.006*"write"


Topic: 4 
Words: 0.009*"line" + 0.008*"subject" + 0.007*"file" + 0.007*"window" + 0.006*"organ" + 0.006*"program" + 0.006*"imag" + 0.005*"write" + 0.004*"post" + 0.004*"wire"


Topic: 5 
Words: 0.010*"peopl" + 0.006*"say" + 0.006*"write" + 0.006*"state" + 0.005*"go" + 0.005*"right" + 0.0

In [49]:
num = 100
unseen_document = newsgroups_test.data[num]
print(unseen_document)

Subject: help
From: C..Doelle@p26.f3333.n106.z1.fidonet.org (C. Doelle)
Lines: 13

Hello All!

    It is my understanding that all True-Type fonts in Windows are loaded in
prior to starting Windows - this makes getting into Windows quite slow if you
have hundreds of them as I do.  First off, am I correct in this thinking -
secondly, if that is the case - can you get Windows to ignore them on boot and
maybe make something like a PIF file to load them only when you enter the
applications that need fonts?  Any ideas?


Chris

 * Origin: chris.doelle.@f3333.n106.z1.fidonet.org (1:106/3333.26)



In [50]:
# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.5670673251152039	 Topic: 0.009*"line" + 0.008*"subject" + 0.007*"file" + 0.007*"window" + 0.006*"organ"
Score: 0.415441632270813	 Topic: 0.015*"line" + 0.014*"subject" + 0.013*"organ" + 0.008*"post" + 0.008*"drive"


In [51]:
print(newsgroups_test.target[num])

2
